# Imports and loading data

In [60]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import os
import random

**Reading Data**

In [61]:
sem1A = pd.read_csv('2017_Sem1_A_batch.csv',error_bad_lines=False, skiprows = 1)
sem1B = pd.read_csv('2017_Sem1Final_Bgroup.csv',error_bad_lines=False)
sem2A = pd.read_csv('_2017_sem2_A - 2017_sem2.csv',error_bad_lines=False, skiprows=1)
sem2B = pd.read_csv('2017_sem2_Bgroup.csv - 2017_sem2.csv',error_bad_lines=False)

## Batch A Semester 1
Functions : removeNames : remove orignal names replacing them with computationally generated names(https://github.com/varangrai/Deep-Learning/tree/master/Anime%20Characters%20Name%20Generation)



PrintCSV : takes csv file and outputs its contents without roll number for privacy reasons

In [62]:
def removeNames(csv1,csv2):
  new_namesA=[]
  new_namesB = []
  with open('untitled.txt', 'r') as f:
    for w in f:
      new_namesB.append(w.strip()[1:-1])
  with open('untitled.txt', 'r') as f:
    for w in f:
      new_namesB.append(w.strip()[1:-1])    
  with open('untitled.txt', 'r') as f:
    for w in f:
      new_namesB.append(w.strip()[1:-1])  
  new_names = new_namesB
  random.shuffle(new_names)    
  newname = new_names[0:csv1.shape[0]]
  cols = list(csv1.columns)
  csv1[str(cols[0])] = newname
   
  newname = new_names[0:csv2.shape[0]]
  cols = list(csv2.columns)
  csv2[str(cols[0])] = newname  
  return csv1,csv2
def printcsv(csv):
  a = list(csv.columns)
  a.pop(1)
  return print(csv[a])
  
sem1A,sem2A = removeNames(sem1A,sem2A)
sem1B,sem2B = removeNames(sem1B,sem2B)    

In [63]:
sem1A.head()

,Sr.No. Name,Roll No.\nCredits,MA101\n4,AP101\n4,AC101\n4,ME101\n4,ME103\n2,HU101\n3,TC\n21,SGPA,Unnamed: 10
0,fuji,2017/A1/01,O,O,O,A,B+,B,21,8.76,NaN
1,kanino,2017/A1/02,O,O,O,A,A,A,21,9.14,NaN
2,bulūcan,2017/A1/03,O,A+,O,A,A,B+,21,8.81,NaN
3,lasdi,2017/A1/04,A+,A+,A,B+,B+,A,21,8.1,NaN
4,sharofa,2017/A1/05,O,O,A+,A,B+,B+,21,8.71,NaN


### **Modify**:
* used to change column names

In [64]:
def modify(df,new_names, cols2drop):  # changing column names
  df = df.drop(cols2drop,axis =1 )
  dicto = dict(zip(list(df.columns),new_names))
  df = df.rename(columns =dicto)
  return df

### **remove_useless**:

 removing students who did not attend exams (please attend exams)

In [65]:
def remove_useless(df):
  for i in df.columns[2:-1]:
    for index,row in df.iterrows():
      if (row[i] != 'O' and row[i] != 'A+' and row[i] != 'A' and row[i] != 'B+' and row[i] != 'B' and row[i] != 'C' and row[i] != 'P' and row[i] != 'F' and row[i] != 'D' and row[i] != 'D+' and row[i] != 'C+'):
        df = df.drop(index = int(index))
  return df 

Changing names applying functions

In [66]:
new_name=['Name', 'Roll No.', 'Math1', 'Physics1', 'Chem', 'BME','Workshop','FEC1','SGPGA1']
todrop = ['TC\n21','Unnamed: 10']
sem1A = modify(sem1A,new_name,todrop)
sem1A = remove_useless(sem1A)
sem1A.head() # our semester 1 A batch data 

,Name,Roll No.,Math1,Physics1,Chem,BME,Workshop,FEC1,SGPGA1
0,fuji,2017/A1/01,O,O,O,A,B+,B,8.76
1,kanino,2017/A1/02,O,O,O,A,A,A,9.14
2,bulūcan,2017/A1/03,O,A+,O,A,A,B+,8.81
3,lasdi,2017/A1/04,A+,A+,A,B+,B+,A,8.1
4,sharofa,2017/A1/05,O,O,A+,A,B+,B+,8.71


## Batch A Semester 2

In [67]:
new_name=['Name', 'Roll No.', 'Math2', 'Physics2', 'EE', 'CO','ED','FEC2','SGPGA2']
todrop = ['TC','Unnamed: 10']
sem2A = modify(sem2A,new_name,todrop)
sem2A = remove_useless(sem2A)
sem2A.head() # our semester 2 batch a data

,Name,Roll No.,Math2,Physics2,EE,CO,ED,FEC2,SGPGA2
1,kanino,2017/A1/01,A+,B+,B+,A+,F,B,6.95
2,bulūcan,2017/A1/02,O,O,A,A+,B,A+,8.9
3,lasdi,2017/A1/03,O,O,A+,A,B,A,8.76
4,sharofa,2017/A1/04,O,A+,A+,A+,B,A+,8.9
5,ka,2017/A1/05,O,O,A,O,P,B+,8.62


## Merging semester 1 and semester 2
We now merge semester files together


In [68]:
def merge_sem(df1,df2):
  list1 = list(df1['Roll No.'])
  list2 = list(df2['Roll No.'])
  list3 = [value for value in list1 if value in list2]
  bl1 = [i in list3 for i in df1['Roll No.']]
  bl2 = [i in list3 for i in df2['Roll No.']]
  new_sem1 = df1[bl1]
  new_sem2 = df2[bl2]
  new_sem1 = new_sem1.reset_index()
  new_sem2 = new_sem2.reset_index()
  new = pd.concat([new_sem1, new_sem2], axis=1)
  new = new.loc[:,~new.columns.duplicated()]
  new =new.drop(['index'], axis=1)
  return new

Printing our A batch results

In [69]:
batch_A_results = merge_sem(sem1A, sem2A)
batch_A_results.head() # semester 1 and 2 a batch combined 

,Name,Roll No.,Math1,Physics1,Chem,BME,Workshop,FEC1,SGPGA1,Math2,Physics2,EE,CO,ED,FEC2,SGPGA2
0,fuji,2017/A1/01,O,O,O,A,B+,B,8.76,A+,B+,B+,A+,F,B,6.95
1,kanino,2017/A1/02,O,O,O,A,A,A,9.14,O,O,A,A+,B,A+,8.9
2,bulūcan,2017/A1/03,O,A+,O,A,A,B+,8.81,O,O,A+,A,B,A,8.76
3,lasdi,2017/A1/04,A+,A+,A,B+,B+,A,8.1,O,A+,A+,A+,B,A+,8.9
4,sharofa,2017/A1/05,O,O,A+,A,B+,B+,8.71,O,O,A,O,P,B+,8.62


### Reordering batch A 

In [70]:
recol=['Name', 'Roll No.', 'Math1', 'Physics1', 'EE', 'CO', 'ED', 'FEC1',
       'SGPGA1', 'Math2', 'Physics2', 'Chem', 'BME', 'Workshop', 'FEC2',
       'SGPGA2']
batch_A_results = batch_A_results[recol]
batch_A_results.head()

,Name,Roll No.,Math1,Physics1,EE,CO,ED,FEC1,SGPGA1,Math2,Physics2,Chem,BME,Workshop,FEC2,SGPGA2
0,fuji,2017/A1/01,O,O,B+,A+,F,B,8.76,A+,B+,O,A,B+,B,6.95
1,kanino,2017/A1/02,O,O,A,A+,B,A,9.14,O,O,O,A,A,A+,8.9
2,bulūcan,2017/A1/03,O,A+,A+,A,B,B+,8.81,O,O,O,A,A,A,8.76
3,lasdi,2017/A1/04,A+,A+,A+,A+,B,A,8.1,O,A+,A,B+,B+,A+,8.9
4,sharofa,2017/A1/05,O,O,A,O,P,B+,8.71,O,O,A+,A,B+,B+,8.62


## Batch B Semester 1

In [71]:
new_name=['Name', 'Roll No.', 'Math1', 'Physics1', 'EE', 'CO','ED','FEC1','SGPGA1']
todrop = ['TC','Unnamed: 10']
sem1B = modify(sem1B,new_name,todrop)
sem1B = remove_useless(sem1B)
sem1B.head() # # our semester 1 batch B data

,Name,Roll No.,Math1,Physics1,EE,CO,ED,FEC1,SGPGA1
0,kow,2017/B1/67,O,A+,O,O,A+,A,9.43
1,jidanek,2017/B1/68,O,O,O,O,A,B,9.24
2,kinaban,2017/B1/69,A+,A,B,A+,B+,B,7.62
3,haga,2017/B1/70,A,O,O,A,B+,B,8.38
4,zark,2017/B1/71,A+,O,O,A,A,B,8.67


## Batch B Semester 2

In [72]:
new_name=['Name', 'Roll No.', 'Math2', 'Physics2', 'Chem', 'BME','Workshop','FEC2','SGPGA2']
todrop = ['TC\n21','Unnamed: 10']
sem2B = modify(sem2B,new_name,todrop)
sem2B = remove_useless(sem2B)
sem2B# our semester 2 batch B data

,Name,Roll No.,Math2,Physics2,Chem,BME,Workshop,FEC2,SGPGA2
0,kow,2017/B1/67,O,O,O,O,O,A,9.71
1,jidanek,2017/B1/68,A+,O,O,A+,B+,A+,9.19
2,kinaban,2017/B1/69,B+,A+,A,B+,A+,B+,7.76
3,haga,2017/B1/70,O,O,O,O,O,A,9.71
4,zark,2017/B1/71,O,A+,O,O,A+,A,9.43
...,...,...,...,...,...,...,...,...,...
1113,fajab,2017/B9/1183,B,B+,B,B+,O,B,6.76
1114,millo,2017/B9/1184,A+,A,B+,B+,O,A,8
1115,hono,2017/B9/1185,B,B+,B+,B,O,B+,6.9
1116,suir,2017/B9/1186,B+,A+,B,B+,B+,A,7.33


Printing our B batch results

In [73]:
batch_B_results=merge_sem(sem1B,sem2B)
temp = merge_sem(sem1B.tail(10), sem2B.tail(10))
temp=pd.concat([batch_B_results, temp])
temp.drop_duplicates(['Roll No.'], keep = 'last', inplace = True)
batch_B_results = temp.reset_index() # merging sem1 b and sem 2 b students data

___
___
___

Concatenating both files together into one to create a final file

In [74]:
Final = pd.concat([batch_B_results,batch_A_results])
Final.reset_index()
Final = Final.drop(['index'], axis = 1)
Final.head() # our B batch data

,Name,Roll No.,Math1,Physics1,EE,CO,ED,FEC1,SGPGA1,Math2,Physics2,Chem,BME,Workshop,FEC2,SGPGA2
0,kow,2017/B1/67,O,A+,O,O,A+,A,9.43,O,O,O,O,O,A,9.71
1,jidanek,2017/B1/68,O,O,O,O,A,B,9.24,A+,O,O,A+,B+,A+,9.19
2,kinaban,2017/B1/69,A+,A,B,A+,B+,B,7.62,B+,A+,A,B+,A+,B+,7.76
3,haga,2017/B1/70,A,O,O,A,B+,B,8.38,O,O,O,O,O,A,9.71
4,zark,2017/B1/71,A+,O,O,A,A,B,8.67,O,A+,O,O,A+,A,9.43


Our subject list

In [75]:
temp = ['Math1', 'Physics1', 'EE', 'CO', 'ED', 'FEC1', 'Math2', 'Physics2', 'Chem', 'BME', 'Workshop', 'FEC2',]

Converting our grades into numeric data

In [76]:
grade2idx = {'F':3,'P':4,'C':5,'B':6,'B+':7,'A':8,'A+':9,'O':10}
for col in temp:
  Final[col]=pd.Series([grade2idx[i] for i in Final[col]])
Final.head()  # converting grades to numerical based grading using grade2idx dictionary

,Name,Roll No.,Math1,Physics1,EE,CO,ED,FEC1,SGPGA1,Math2,Physics2,Chem,BME,Workshop,FEC2,SGPGA2
0,kow,2017/B1/67,10,9,10,10,9,8,9.43,10,10,10,10,10,8,9.71
1,jidanek,2017/B1/68,10,10,10,10,8,6,9.24,9,10,10,9,7,9,9.19
2,kinaban,2017/B1/69,9,8,6,9,7,6,7.62,7,9,8,7,9,7,7.76
3,haga,2017/B1/70,8,10,10,8,7,6,8.38,10,10,10,10,10,8,9.71
4,zark,2017/B1/71,9,10,10,8,8,6,8.67,10,9,10,10,9,8,9.43


Calculating our CGPA

In [77]:
CGPA1=(Final['Math1'])+(Final['Physics1'])+(Final['EE'])+(Final['CO'])+(Final['ED'])/2+(Final['FEC1'])/2
CGPA1=CGPA1/5
Final['SGPGA1']=CGPA1
CGPA2=(Final['Math2'])+(Final['Physics2'])+(Final['Chem'])+(Final['BME'])+(Final['Workshop'])/2+(Final['FEC2'])/2
CGPA2=CGPA2/5
Final['SGPGA2']=CGPA2
Final.head()# final data after combining everything 

,Name,Roll No.,Math1,Physics1,EE,CO,ED,FEC1,SGPGA1,Math2,Physics2,Chem,BME,Workshop,FEC2,SGPGA2
0,kow,2017/B1/67,10,9,10,10,9,8,9.5,10,10,10,10,10,8,9.8
1,jidanek,2017/B1/68,10,10,10,10,8,6,9.4,9,10,10,9,7,9,9.2
2,kinaban,2017/B1/69,9,8,6,9,7,6,7.7,7,9,8,7,9,7,7.8
3,haga,2017/B1/70,8,10,10,8,7,6,8.5,10,10,10,10,10,8,9.8
4,zark,2017/B1/71,9,10,10,8,8,6,8.8,10,9,10,10,9,8,9.5


## Saving the data

In [78]:
Final.to_csv('/content/final.csv',index = False) # converting everything to csv file for use

In [81]:
dfp = pd.read_csv('/content/16Final.csv' )

dfp.head()
vp=pd.concat([Final,dfp])
vp.reset_index()
vp.head()

vp.to_csv('/content/USEME.csv',index = False)

In [ ]:
vp1 = vp.drop(columns = [vp.columns[1]])
vp1.to_csv('vp1.csv',index=False)

In [ ]:
vp.reset_index()
year = []
for index,rows in vp.iterrows():
  year.append(str(rows['Roll No.']).split('/')[0])
vp2=vp
vp2['Roll No.'] = year 


In [ ]:
vp2.rename(columns = {'SGPA1': 'CGPA1','SGPA2':'CGPA2','Roll No.': 'Year'},inplace = True)
vp2.to_csv('vp2.csv',index= False)